# Podatkovni model za aplikacijo

Za potrebe spletne aplikacije smo pripravili ogrodje podatkovnega modela, ki skrbi za komunikacijo z relacijsko podatkovno bazo in predstavitev entitet v bazi z objekti. Takemu pristopu pravimo *objektno-relacijsko preslikovanje* (*ORM*, angl. *object-relational mapping*).

In [ ]:
from model import Kraj, Oseba, Racun, Transakcija, vzpostavi_povezavo, ustvari_bazo, uvozi_podatke
conn = vzpostavi_povezavo()

Pripravimo si prvi objekt in ga shranimo v bazo.

In [ ]:
lj = Kraj.ustvari(1000, 'Ljubljana')
lj.shrani()

Izpišimo seznam krajev iz baze.

In [ ]:
list(Kraj.seznam())

Pripravimo še ostale objekte. Pri shranjevanju v bazo se bodo shranili tudi povezani objekti.

In [ ]:
mb = Kraj.ustvari(2000, 'Maribor')

janez = Oseba.ustvari('Janez', 'Novak', '1234567890123', 'Tržaška cesta', lj)
micka = Oseba.ustvari('Micka', 'Kovač', '9876543210987', 'Ljubljanska cesta', mb)
peter = Oseba.ustvari('Peter', 'Klepec', '1029384756601', 'Klepčeva ulica', 1000)

racun_j1 = Racun.ustvari(oseba_id=janez)
racun_m = Racun.ustvari(oseba_id=micka)
racun_j2 = Racun.ustvari(oseba_id=janez)
racun_p = Racun.ustvari(oseba_id=peter)

with conn.transaction():
    Transakcija.ustvari(racun_id=racun_j1, znesek=100, opis='odprtje računa').shrani()
    Transakcija.ustvari(racun_id=racun_m, znesek=500, opis='prihranki').shrani()
    Transakcija.ustvari(racun_id=racun_j2, znesek=10, opis='varčevanje').shrani()
    Transakcija.ustvari(racun_id=racun_j1, znesek=-15, opis='dvig').shrani()
    Transakcija.ustvari(racun_id=racun_p, znesek=1000).shrani()

In [ ]:
list(Oseba.seznam())

In [ ]:
list(Transakcija.seznam())

Do vrednostih posameznih atributov v objektih dostopamo z istoimenskimi polji. Če gre za referenco, bomo tako dobili ustrezen objekt.

In [ ]:
peter.naslov

In [ ]:
peter.kraj_id

Z indeksiranjem dobimo vrednosti iz baze - tj., v primeru referenc dobimo ustrezno vrednost ključa.

In [ ]:
tr = Transakcija.z_id(5)
tr['cas']

In [ ]:
tr['racun_id']

Vrednosti lahko tudi spreminjamo ter nato spremembe shranimo v bazo.

In [ ]:
tr.opis = 'dobitek na lotu'
tr.shrani()

In [ ]:
for tid, znesek, racun, cas, opis in Transakcija.seznam():
    print(f"{tid}: znesek {znesek} na računu {racun} osebe {Racun.z_id(racun).oseba_id}: {opis} ({cas})")

**Pozor**: tak način ni učinkovit, saj se pri vsaki transakciji izvedeta še poizvedbi za račun in za osebo. Če bomo v aplikaciji želeli tak izpis, bomo implementirali ustrezno metodo, ki bo iz baze pridobila želene podatke s čim manjšim številom poizvedb.

## Uvoz podatkov iz datotek

Pobrišimo tabele in jih napolnimo s podatki, ki smo jih generirali s spletnim orodjem [Mockaroo](https://www.mockaroo.com).

In [ ]:
ustvari_bazo(izbrisi=True)
uvozi_podatke()

Oglejmo si nekaj uvoženih podatkov.

In [ ]:
for _, oseba in zip(range(10), Oseba.seznam()):
    print(oseba)